## HW 11 - Data fitting with various functions - Due Monday Nov 8, 2021


Dowload the data from the SDSS DR16:<br>
https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.fits
NOTE: it is a large file and will take a while.


Using masking. Select the approprate fitting function and fit the following selections of the data:
1. Select stars only with with $-1< GLAT < 1$ <br>
   __Fit TEFF,  (TEFF\_ERR)  vs.  LogG (LOGG\_ERR)__<br>
<br>
2. Select stars only with with $GLAT < -10$ or $GLAT > 10$ <br>
   __Fit {\bf  Fe\_H,  (Fe\_H\_ERR)  vs.  O\_FE (O\_FE\_ERR)__<br>
<br>
3. Select stars only with with $-2< GLAT < 2$ <br>
   __Fit {\bf  Fe\_H,  (Fe\_H\_ERR)  vs.  O\_FE (O\_FE\_ERR)__<br>
<br>
4. Select stars only with with $GLAT < -10$ or $GLAT > 10$ <br>
   __Fit {\bf  TEFF,  (TEFF\_ERR)   vs.  (J - K) (J\_ERR, K\_ERR)__<br>
<br>
5. Select stars only with with $-10< GLAT < 10$<br>
   __Fit {\bf  GLON  vs.  VHEILO\_AVG (VERR)__<br>
<br>
6. __Fit {\bf NVISITS vs.  VSCATTER (VERR)__ For NVISITS $>= 1$<br>
<br>
7. __Fit {\bf  Fe\_H,  (Fe\_H\_ERR)  vs.  NI\_FE (NI\_FE\_ERR)__<br>
<br>

For each of the above selections:

1. What are the best-fit values of the parameters? 
2. Which fucntion fits the data best?
3. Is there a second population? (Can you fit outliers with a reasonable linear trend)

In [2]:
from matplotlib import pyplot as plt
import numpy as np
from astropy.io import fits 
import scipy.optimize as opt

# POTENTIAL FITTING FUNCTIONS
def linear(x,m,b):
    return m*x+b

def poly2(x,a,b,c):
    return a*(x**2)+(b*x)+c

def poly3(x,a,b,c,d):
    return (a*(x**3))+(b*(x**2))+(c*x)+d

def exponential(x,a,b,c,d):
    return a*np.exp(b*x + c) + d

def cosfit (x,a,b,c,d):
    return a*np.cos(b*x + c) + d

def logfit(x,a,b,c,d):
    return a*np.log(b*x + c) + d

def gausian(x,mu,sigma):
    part1 = (1.0/np.sqrt(2*np.pi*sigma**2))
    return (part1*(np.exp((-1*x - mu)**2/(2*sigma**2))))

def lorentzian_cauchy (x,a,b,c):
    return (c)*(b**2/((x-a)**2 + b**2))

def sigmoid(x,a,b,c,d):
    return (c/(1 + np.exp(-b*(x-a))) + d)

# READ IN FITS FILES
star_hdus = fits.open('allStar-r12-l33.fits')
star = star_hdus[1].data
print(star_hdus[1].columns)
star_hdus.close()

# TWO BITWISE FLAGS FOR BAD DATA             
badbits = 2**23        # aspcapstar flag - Chemistry
suspectbits = 2**16    # star flag - Stellar parameters



ColDefs(
    name = 'APSTAR_ID'; format = '57A'
    name = 'TARGET_ID'; format = '47A'
    name = 'ASPCAP_ID'; format = '59A'
    name = 'FILE'; format = '39A'
    name = 'APOGEE_ID'; format = '18A'
    name = 'TELESCOPE'; format = '8A'
    name = 'LOCATION_ID'; format = 'J'
    name = 'FIELD'; format = '16A'
    name = 'J'; format = 'E'
    name = 'J_ERR'; format = 'E'
    name = 'H'; format = 'E'
    name = 'H_ERR'; format = 'E'
    name = 'K'; format = 'E'
    name = 'K_ERR'; format = 'E'
    name = 'RA'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'GLON'; format = 'D'
    name = 'GLAT'; format = 'D'
    name = 'APOGEE_TARGET1'; format = 'J'
    name = 'APOGEE_TARGET2'; format = 'J'
    name = 'APOGEE_TARGET3'; format = 'J'
    name = 'APOGEE2_TARGET1'; format = 'J'
    name = 'APOGEE2_TARGET2'; format = 'J'
    name = 'APOGEE2_TARGET3'; format = 'J'
    name = 'TARGFLAGS'; format = '192A'
    name = 'SURVEY'; format = '35A'
    name = 'PROGRAMNAME'; format = '18A'
    na

In [3]:
def mcFit2(func, x, y, x_err, y_err):
    slope = list()
    y_ints = list()
    iters = 500 
    for i in range(iters):
        # remember random normal distribution (Gaussian)
        weightsx = np.random.randn(len(y))
        weightsy = np.random.randn(len(y))

        y_adj = y + y_err*weightsy
        x_adj = x + x_err*weightsx 

        popt, pcov = opt.curve_fit(func, x_adj, y_adj)
        slope.append(popt[0])
        y_ints.append(popt[1])

    return (np.median(slope),np.median(y_ints))


def mcFit3(func, x, y, x_err, y_err,p0=[1,1,1]):
    paramA = list()
    paramB = list()
    paramC = list()
    iters = 500 
    for i in range(iters):
        # remember random normal distribution (Gaussian)
        weightsx = np.random.randn(len(y))
        weightsy = np.random.randn(len(y))

        y_adj = y + y_err*weightsy
        x_adj = x + x_err*weightsx 

        popt, pcov = opt.curve_fit(func, x_adj, y_adj, p0=p0)
        paramA.append(popt[0])
        paramB.append(popt[1])
        paramC.append(popt[2])
    return (np.median(paramA),np.median(paramB),np.median(paramC))

def mcFit4(func, x, y, x_err, y_err, p0=[1,1,1,1]):
    paramA = list()
    paramB = list()
    paramC = list()
    paramD = list()
    iters = 500
    for i in range(iters):
        # remember random normal distribution (Gaussian)
        weightsx = np.random.randn(len(y))
        weightsy = np.random.randn(len(y))

        y_adj = y + y_err*weightsy
        x_adj = x + x_err*weightsx 

        popt, pcov = opt.curve_fit(func, x_adj, y_adj, p0=p0)
        paramA.append(popt[0])
        paramB.append(popt[1])
        paramC.append(popt[2])
        paramD.append(popt[3])

    return (np.median(paramA),np.median(paramB),np.median(paramC),np.median(paramD))

In [ ]:
# Make a Boolena Mask to remove bad data
gd = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) 
good = np.where(gd)[0]



# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig = plt.figure(figsize=(20,15))
ax = fig.add_subplot(111)
ax.scatter(star['RA'][good], star['DEC'][good], s=1, c='b', alpha=0.1)

ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
plt.show()

In [ ]:
#PART #1:
#Select stars only with with  −1 < 𝐺𝐿𝐴𝑇 < 1 
#Fit TEFF, (TEFF_ERR) vs. LogG (LOGG_ERR)

temp_mask1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['TEFF'] > -9000) & (star['LOGG'] > -9000) &\
     ((star['GLAT'] > -10) | (star['GLAT'] < 10))
mask1 = np.where(temp_mask1)[0]

xdat1 = star['TEFF'][mask1]
ydat1 = star['LOGG'][mask1]
xerr1 = star['TEFF_ERR'][mask1]
yerr1 = star['LOGG_ERR'][mask1]

## LINEAR FIT
#slope, intercept = mcFit2(linear, xdat1, ydat1, xerr1, yerr1)
#print("LINEAR: ", slope, intercept)

## POLY2
#poly2_prA, poly2_prB, poly2_prC = mcFit3(poly2, xdat1, ydat1, xerr1, yerr1)
#print("POLY2: ", poly2_prA, poly2_prB, poly2_prC)

## POLY3
#poly3_prA, poly3_prB, poly3_prC, poly3_prD = mcFit4(poly3, xdat1, ydat1, xerr1, yerr1)
#print("POLY3: ", poly3_prA, poly3_prB, poly3_prC, poly3_prD)

## EXPONENTIAL
#exp_prA, exp_prB, exp_prC, exp_prD = mcFit4(exponential, xdat1, ydat1, xerr1, yerr1)
#print("EXPONENTIAL: ", exp_prA, exp_prB, exp_prC, exp_prD)

## COSFITS
#prA, prB, prC, prD = mcFit4(cosfit, xdat1, ydat1, xerr1, yerr1)
#print("COSFIT: ",prA, prB, prC, prD)

##LOGFIT
#log_prA, log_prB, log_prC, log_prD = mcFit4(logfit, xdat1, ydat1, xerr1, yerr1)
#print("LOGFIT: ",log_prA, log_prB, log_prC, log_prD)

##GAUSIAN FIT
#gausian_prA, gausian_prB = mcFit2(gausian, xdat1, ydat1, xerr1, yerr1)
#print("GAUSIAN: ", gausian_prA, gausian_prB)

##LORENTZIAN-CAUCHY FIT
#lc_prA, lc_prB, lc_prC = mcFit3(lorentzian_cauchy, xdat1, ydat1, xerr1, yerr1)#, p0=[0,633.7034807407312,3494.755148226808])
#print("LORENTZIAN-CAUCHY: ", lc_prA, lc_prB, lc_prC)

##SIGMOID FIT
#sig_prA, sig_prB, sig_prC, sig_prD = mcFit4(sigmoid, xdat1, ydat1, xerr1, yerr1)
#print("SIGMOID: ", sig_prA, sig_prB, sig_prC, sig_prD)

# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

# PLOT DATA AND ERROR BARS
ax1.scatter(xdat1,ydat1,s=1,c='b',alpha=0.1)
ax1.errorbar(xdat1, ydat1, xerr=xerr1, yerr=yerr1, ecolor='grey',fmt='none', capsize=5, zorder=0)

x_plot = np.linspace(3000,10000,1000) # X-PLOTING FOR FITS
##linear fit
#ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')

##poly2 fit
#ax1.plot(x_plot,poly2(x_plot, poly2_prA, poly2_prB, poly2_prC), label='poly2: FIT')

##poly3 fit
#ax1.plot(x_plot,poly3(x_plot, poly3_prA, poly3_prB, poly3_prC, poly3_prD), label='poly3: FIT')

##exponential fit
#ax1.plot(x_plot,exponential(x_plot, exp_prA, exp_prB, exp_prC, exp_prD), label='exponential: FIT')

##cosfit
#ax1.plot(x_plot,cosfit(x_plot, prA, prB, prC, prD), label='cosfit: FIT')

##logfit 
#ax1.plot(x_plot,logfit(x_plot, log_prA, log_prB, log_prC, log_prD), label='logfit: FIT')

##guasian fit
#ax1.plot(x_plot,gausian(x_plot, gausian_prA, gausian_prB), label='gausian: FIT')

##lorentzian-cauchy fit
#ax1.plot(x_plot,lorentzian_cauchy(x_plot, lc_prA, lc_prB, lc_prC), label='lorentzian-cauchy: FIT')

##sigmoid fit
#ax1.plot(x_plot,sigmoid(x_plot, sig_prA, sig_prB, sig_prC, sig_prD), label='sigmoid: FIT')



ax1.set_xlabel('$TEFF$', fontsize=18)
ax1.set_ylabel('$LOGG$', fontsize=18)
plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
#PART 2
#Select stars only with with  𝐺𝐿𝐴𝑇<−10  or  𝐺𝐿𝐴𝑇>10 
#Fit {\bf Fe_H, (Fe_H_ERR) vs. O_FE (O_FE_ERR)

temp_mask1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['O_FE'] > -9000) & (star['FE_H'] > -9000) &\
     ((star['GLAT'] > -10) | (star['GLAT'] < 10))
mask1 = np.where(temp_mask1)[0]

xdat1 = star['FE_H'][mask1]
ydat1 = star['O_FE'][mask1]
xerr1 = star['FE_H_ERR'][mask1]
yerr1 = star['O_FE_ERR'][mask1]


## LINEAR FIT
#slope, intercept = mcFit2(linear, xdat1, ydat1, xerr1, yerr1)
#print("LINEAR: ", slope, intercept)

## POLY2
#poly2_prA, poly2_prB, poly2_prC = mcFit3(poly2, xdat1, ydat1, xerr1, yerr1)
#print("POLY2: ", poly2_prA, poly2_prB, poly2_prC)

## POLY3
#poly3_prA, poly3_prB, poly3_prC, poly3_prD = mcFit4(poly3, xdat1, ydat1, xerr1, yerr1)
#print("POLY3: ", poly3_prA, poly3_prB, poly3_prC, poly3_prD)

## EXPONENTIAL
#exp_prA, exp_prB, exp_prC, exp_prD = mcFit4(exponential, xdat1, ydat1, xerr1, yerr1)
#print("EXPONENTIAL: ", exp_prA, exp_prB, exp_prC, exp_prD)

## COSFITS
#prA, prB, prC, prD = mcFit4(cosfit, xdat1, ydat1, xerr1, yerr1)
#print("COSFIT: ",prA, prB, prC, prD)

##LOGFIT
#log_prA, log_prB, log_prC, log_prD = mcFit4(logfit, xdat1, ydat1, xerr1, yerr1)
#print("LOGFIT: ",log_prA, log_prB, log_prC, log_prD)

##GAUSIAN FIT
#gausian_prA, gausian_prB = mcFit2(gausian, xdat1, ydat1, xerr1, yerr1)
#print("GAUSIAN: ", gausian_prA, gausian_prB)

##LORENTZIAN-CAUCHY FIT
#lc_prA, lc_prB, lc_prC = mcFit3(lorentzian_cauchy, xdat1, ydat1, xerr1, yerr1)
#print("LORENTZIAN-CAUCHY: ", lc_prA, lc_prB, lc_prC)

##SIGMOID FIT
#sig_prA, sig_prB, sig_prC, sig_prD = mcFit4(sigmoid, xdat1, ydat1, xerr1, yerr1)
#print("SIGMOID: ", sig_prA, sig_prB, sig_prC, sig_prD)

# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(122)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

# PLOT DATA AND ERROR BARS
ax1.scatter(xdat1,ydat1,s=1,c='b',alpha=0.1)
#ax1.errorbar(xdat1, ydat1, xerr=xerr1, yerr=yerr1, ecolor='grey',fmt='none', capsize=5, zorder=0)

x_plot = np.linspace(-2.5,0.5,1000) # X-PLOTING FOR FITS

##linear fit
#ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')

##poly2 fit
#ax1.plot(x_plot,poly2(x_plot, poly2_prA, poly2_prB, poly2_prC), label='poly2: FIT')

##poly3 fit
#ax1.plot(x_plot,poly3(x_plot, poly3_prA, poly3_prB, poly3_prC, poly3_prD), label='poly3: FIT')

##exponential fit
#ax1.plot(x_plot,exponential(x_plot, exp_prA, exp_prB, exp_prC, exp_prD), label='exponential: FIT')

##cosfit
#ax1.plot(x_plot,cosfit(x_plot, prA, prB, prC, prD), label='cosfit: FIT')

##logfit 
#ax1.plot(x_plot,logfit(x_plot, log_prA, log_prB, log_prC, log_prD), label='logfit: FIT')

##guasian fit
#ax1.plot(x_plot,gausian(x_plot, gausian_prA, gausian_prB), label='gausian: FIT')

##lorentzian-cauchy fit
#ax1.plot(x_plot,lorentzian_cauchy(x_plot, lc_prA, lc_prB, lc_prC), label='lorentzian-cauchy: FIT')

##sigmoid fit
#ax1.plot(x_plot,sigmoid(x_plot, sig_prA, sig_prB, sig_prC, sig_prD), label='sigmoid: FIT')



ax1.set_xlabel('$FE_H$', fontsize=18)
ax1.set_ylabel('$O_{FE}$', fontsize=18)
#plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
##PART 3
#Select stars only with with  −2<𝐺𝐿𝐴𝑇<2 
#Fit {\bf Fe_H, (Fe_H_ERR) vs. O_FE (O_FE_ERR)

temp_mask1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['O_FE'] > -9000) & (star['FE_H'] > -9000) &\
     ((star['GLAT'] > -2) | (star['GLAT'] < 2))
mask1 = np.where(temp_mask1)[0]

xdat1 = star['FE_H'][mask1]
ydat1 = star['O_FE'][mask1]
xerr1 = star['FE_H_ERR'][mask1]
yerr1 = star['O_FE_ERR'][mask1]


## LINEAR FIT
#slope, intercept = mcFit2(linear, xdat1, ydat1, xerr1, yerr1)
#print("LINEAR: ", slope, intercept)

## POLY2
#poly2_prA, poly2_prB, poly2_prC = mcFit3(poly2, xdat1, ydat1, xerr1, yerr1)
#print("POLY2: ", poly2_prA, poly2_prB, poly2_prC)

## POLY3
#poly3_prA, poly3_prB, poly3_prC, poly3_prD = mcFit4(poly3, xdat1, ydat1, xerr1, yerr1)
#print("POLY3: ", poly3_prA, poly3_prB, poly3_prC, poly3_prD)

## EXPONENTIAL
#exp_prA, exp_prB, exp_prC, exp_prD = mcFit4(exponential, xdat1, ydat1, xerr1, yerr1)
#print("EXPONENTIAL: ", exp_prA, exp_prB, exp_prC, exp_prD)

## COSFITS
#prA, prB, prC, prD = mcFit4(cosfit, xdat1, ydat1, xerr1, yerr1)
#print("COSFIT: ",prA, prB, prC, prD)

##LOGFIT
#log_prA, log_prB, log_prC, log_prD = mcFit4(logfit, xdat1, ydat1, xerr1, yerr1)
#print("LOGFIT: ",log_prA, log_prB, log_prC, log_prD)

##GAUSIAN FIT
#gausian_prA, gausian_prB = mcFit2(gausian, xdat1, ydat1, xerr1, yerr1)
#print("GAUSIAN: ", gausian_prA, gausian_prB)

##LORENTZIAN-CAUCHY FIT
#lc_prA, lc_prB, lc_prC = mcFit3(lorentzian_cauchy, xdat1, ydat1, xerr1, yerr1)
#print("LORENTZIAN-CAUCHY: ", lc_prA, lc_prB, lc_prC)

##SIGMOID FIT
#sig_prA, sig_prB, sig_prC, sig_prD = mcFit4(sigmoid, xdat1, ydat1, xerr1, yerr1)
#print("SIGMOID: ", sig_prA, sig_prB, sig_prC, sig_prD)

# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(122)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

# PLOT DATA AND ERROR BARS
ax1.scatter(xdat1,ydat1,s=1,c='b',alpha=0.1)
#ax1.errorbar(xdat1, ydat1, xerr=xerr1, yerr=yerr1, ecolor='grey',fmt='none', capsize=5, zorder=0)

x_plot = np.linspace(-1.5,1,1000) # X-PLOTING FOR FITS

##linear fit
#ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')

##poly2 fit
#ax1.plot(x_plot,poly2(x_plot, poly2_prA, poly2_prB, poly2_prC), label='poly2: FIT')

##poly3 fit
#ax1.plot(x_plot,poly3(x_plot, poly3_prA, poly3_prB, poly3_prC, poly3_prD), label='poly3: FIT')

##exponential fit
#ax1.plot(x_plot,exponential(x_plot, exp_prA, exp_prB, exp_prC, exp_prD), label='exponential: FIT')

##cosfit
#ax1.plot(x_plot,cosfit(x_plot, prA, prB, prC, prD), label='cosfit: FIT')

##logfit 
#ax1.plot(x_plot,logfit(x_plot, log_prA, log_prB, log_prC, log_prD), label='logfit: FIT')

##guasian fit
#ax1.plot(x_plot,gausian(x_plot, gausian_prA, gausian_prB), label='gausian: FIT')

##lorentzian-cauchy fit
#ax1.plot(x_plot,lorentzian_cauchy(x_plot, lc_prA, lc_prB, lc_prC), label='lorentzian-cauchy: FIT')

##sigmoid fit
#ax1.plot(x_plot,sigmoid(x_plot, sig_prA, sig_prB, sig_prC, sig_prD), label='sigmoid: FIT')



ax1.set_xlabel('$FE_H$', fontsize=18)
ax1.set_ylabel('$O_FE$', fontsize=18)
plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
##PART 4
#Select stars only with with  𝐺𝐿𝐴𝑇<−10  or  𝐺𝐿𝐴𝑇>10 
#Fit {\bf TEFF, (TEFF_ERR) vs. (J - K) (J_ERR, K_ERR)

temp_mask1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['TEFF'] > -9000) & (star['LOGG'] > -9000) &\
     ((star['GLAT'] > -10) | (star['GLAT'] < 10))
mask1 = np.where(temp_mask1)[0]

temp_dat = star['J'] - star['K']
temp_err = star['J_ERR'] - star['K_ERR']

xdat1 = star['TEFF'][mask1]
ydat1 = temp_dat[mask1]
xerr1 = star['TEFF_ERR'][mask1]
yerr1 = temp_err[mask1]

## LINEAR FIT
#slope, intercept = mcFit2(linear, xdat1, ydat1, xerr1, yerr1)
#print("LINEAR: ", slope, intercept)

## POLY2
#poly2_prA, poly2_prB, poly2_prC = mcFit3(poly2, xdat1, ydat1, xerr1, yerr1)
#print("POLY2: ", poly2_prA, poly2_prB, poly2_prC)

## POLY3
#poly3_prA, poly3_prB, poly3_prC, poly3_prD = mcFit4(poly3, xdat1, ydat1, xerr1, yerr1)
#print("POLY3: ", poly3_prA, poly3_prB, poly3_prC, poly3_prD)

## EXPONENTIAL
#exp_prA, exp_prB, exp_prC, exp_prD = mcFit4(exponential, xdat1, ydat1, xerr1, yerr1)
#print("EXPONENTIAL: ", exp_prA, exp_prB, exp_prC, exp_prD)

## COSFITS
#prA, prB, prC, prD = mcFit4(cosfit, xdat1, ydat1, xerr1, yerr1)
#print("COSFIT: ",prA, prB, prC, prD)

##LOGFIT
#log_prA, log_prB, log_prC, log_prD = mcFit4(logfit, xdat1, ydat1, xerr1, yerr1)
#print("LOGFIT: ",log_prA, log_prB, log_prC, log_prD)

##GAUSIAN FIT
#gausian_prA, gausian_prB = mcFit2(gausian, xdat1, ydat1, xerr1, yerr1)
#print("GAUSIAN: ", gausian_prA, gausian_prB)

##LORENTZIAN-CAUCHY FIT
#c_prA, lc_prB, lc_prC = mcFit3(lorentzian_cauchy, xdat1, ydat1, xerr1, yerr1)#, p0=[0,633.7034807407312,3494.755148226808])
#print("LORENTZIAN-CAUCHY: ", lc_prA, lc_prB, lc_prC)

##SIGMOID FIT
#sig_prA, sig_prB, sig_prC, sig_prD = mcFit4(sigmoid, xdat1, ydat1, xerr1, yerr1)
#print("SIGMOID: ", sig_prA, sig_prB, sig_prC, sig_prD)

# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(133)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

# PLOT DATA AND ERROR BARS
ax1.scatter(xdat1,ydat1,s=1,c='b',alpha=0.1)
ax1.errorbar(xdat1, ydat1, xerr=xerr1, yerr=yerr1, ecolor='grey',fmt='none', capsize=5, zorder=0)


x_plot = np.linspace(3000,10000,1000) # X-PLOTING FOR FITS
##linear fit
#ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')

##poly2 fit
#ax1.plot(x_plot,poly2(x_plot, poly2_prA, poly2_prB, poly2_prC), label='poly2: FIT')
ax1.plot(x_plot,poly2(x_plot, 2.060724302965711e-07, -0.0025112238316073255, 7.8434516668057395), label='poly2: FIT')

##poly3 fit
#ax1.plot(x_plot,poly3(x_plot, poly3_prA, poly3_prB, poly3_prC, poly3_prD), label='poly3: FIT')

##exponential fit
#ax1.plot(x_plot,exponential(x_plot, exp_prA, exp_prB, exp_prC, exp_prD), label='exponential: FIT')

##cosfit
#ax1.plot(x_plot,cosfit(x_plot, prA, prB, prC, prD), label='cosfit: FIT')

##logfit 
#ax1.plot(x_plot,logfit(x_plot, log_prA, log_prB, log_prC, log_prD), label='logfit: FIT')

##guasian fit
#ax1.plot(x_plot,gausian(x_plot, gausian_prA, gausian_prB), label='gausian: FIT')

##lorentzian-cauchy fit
#ax1.plot(x_plot,lorentzian_cauchy(x_plot, lc_prA, lc_prB, lc_prC), label='lorentzian-cauchy: FIT')

##sigmoid fit
#ax1.plot(x_plot,sigmoid(x_plot, sig_prA, sig_prB, sig_prC, sig_prD), label='sigmoid: FIT')


ax1.set_ylim(-1, 7.5)
ax1.set_xlabel('$TEFF$', fontsize=18)
ax1.set_ylabel('$J - K$', fontsize=18)
plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
##PART 5
#Select stars only with with  −10<𝐺𝐿𝐴𝑇<10 
#Fit {\bf GLON vs. VHEILO_AVG (VERR)

temp_mask1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['TEFF'] > -9000) & (star['LOGG'] > -9000) &\
     ((star['GLAT'] > -10) | (star['GLAT'] < 10))
mask1 = np.where(temp_mask1)[0]

xdat1 = star['GLON'][mask1]
ydat1 = star['VHELIO_AVG'][mask1]
xerr1 = xdat1 / 1000
yerr1 = star['VERR'][mask1]

## LINEAR FIT
#slope, intercept = mcFit2(linear, xdat1, ydat1, xerr1, yerr1)
#print("LINEAR: ", slope, intercept)

## POLY2
#poly2_prA, poly2_prB, poly2_prC = mcFit3(poly2, xdat1, ydat1, xerr1, yerr1)
#print("POLY2: ", poly2_prA, poly2_prB, poly2_prC)

## POLY3
#poly3_prA, poly3_prB, poly3_prC, poly3_prD = mcFit4(poly3, xdat1, ydat1, xerr1, yerr1)
#print("POLY3: ", poly3_prA, poly3_prB, poly3_prC, poly3_prD)

## EXPONENTIAL
#exp_prA, exp_prB, exp_prC, exp_prD = mcFit4(exponential, xdat1, ydat1, xerr1, yerr1)
#print("EXPONENTIAL: ", exp_prA, exp_prB, exp_prC, exp_prD)

## COSFITS
#prA, prB, prC, prD = mcFit4(cosfit, xdat1, ydat1, xerr1, yerr1)
#print("COSFIT: ",prA, prB, prC, prD)

##LOGFIT
#log_prA, log_prB, log_prC, log_prD = mcFit4(logfit, xdat1, ydat1, xerr1, yerr1)
#print("LOGFIT: ",log_prA, log_prB, log_prC, log_prD)

##GAUSIAN FIT
#gausian_prA, gausian_prB = mcFit2(gausian, xdat1, ydat1, xerr1, yerr1)
#print("GAUSIAN: ", gausian_prA, gausian_prB)

##LORENTZIAN-CAUCHY FIT
#lc_prA, lc_prB, lc_prC = mcFit3(lorentzian_cauchy, xdat1, ydat1, xerr1, yerr1)#, p0=[0,633.7034807407312,3494.755148226808])
#print("LORENTZIAN-CAUCHY: ", lc_prA, lc_prB, lc_prC)

##SIGMOID FIT
#sig_prA, sig_prB, sig_prC, sig_prD = mcFit4(sigmoid, xdat1, ydat1, xerr1, yerr1)
#print("SIGMOID: ", sig_prA, sig_prB, sig_prC, sig_prD)

# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

# PLOT DATA AND ERROR BARS
ax1.scatter(xdat1,ydat1,s=1,c='b',alpha=0.1)
ax1.errorbar(xdat1, ydat1, yerr=yerr1, ecolor='grey',fmt='none', capsize=5, zorder=0)

x_plot = np.linspace(0,360,1000) # X-PLOTING FOR FITS

##linear fit
#ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')

##poly2 fit
#ax1.plot(x_plot,poly2(x_plot, poly2_prA, poly2_prB, poly2_prC), label='poly2: FIT')

##poly3 fit
#ax1.plot(x_plot,poly3(x_plot, poly3_prA, poly3_prB, poly3_prC, poly3_prD), label='poly3: FIT')

##exponential fit
#ax1.plot(x_plot,exponential(x_plot, exp_prA, exp_prB, exp_prC, exp_prD), label='exponential: FIT')

##cosfit
#ax1.plot(x_plot,cosfit(x_plot, prA, prB, prC, prD), label='cosfit: FIT')
ax1.plot(x_plot,cosfit(x_plot, -50., 0.01, 6.243681807922574, -2.3918461950214036), label='cosfit: FIT')


##logfit 
#ax1.plot(x_plot,logfit(x_plot, log_prA, log_prB, log_prC, log_prD), label='logfit: FIT')

##guasian fit
#ax1.plot(x_plot,gausian(x_plot, gausian_prA, gausian_prB), label='gausian: FIT')

##lorentzian-cauchy fit
#ax1.plot(x_plot,lorentzian_cauchy(x_plot, lc_prA, lc_prB, lc_prC), label='lorentzian-cauchy: FIT')

##sigmoid fit
#ax1.plot(x_plot,sigmoid(x_plot, sig_prA, sig_prB, sig_prC, sig_prD), label='sigmoid: FIT')



ax1.set_xlabel('$GLON$', fontsize=18)
ax1.set_ylabel('$VHEILO$ $AVG$', fontsize=18)
plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
##PART 6
#Fit {\bf NVISITS vs. VSCATTER (VERR) For NVISITS  >=1 

temp_mask1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['TEFF'] > -9000) & (star['LOGG'] > -9000) &\
     (star['NVISITS'] >= 1)
mask1 = np.where(temp_mask1)[0]

xdat1 = star['NVISITS'][mask1]
ydat1 = star['VSCATTER'][mask1]
xerr1 = xdat1 / 1000
yerr1 = star['VERR'][mask1]

## LINEAR FIT
slope, intercept = mcFit2(linear, xdat1, ydat1, xerr1, yerr1)
print("LINEAR: ", slope, intercept)

## POLY2
#poly2_prA, poly2_prB, poly2_prC = mcFit3(poly2, xdat1, ydat1, xerr1, yerr1)
#print("POLY2: ", poly2_prA, poly2_prB, poly2_prC)

## POLY3
#poly3_prA, poly3_prB, poly3_prC, poly3_prD = mcFit4(poly3, xdat1, ydat1, xerr1, yerr1)
#print("POLY3: ", poly3_prA, poly3_prB, poly3_prC, poly3_prD)

## EXPONENTIAL
#exp_prA, exp_prB, exp_prC, exp_prD = mcFit4(exponential, xdat1, ydat1, xerr1, yerr1)
#print("EXPONENTIAL: ", exp_prA, exp_prB, exp_prC, exp_prD)

## COSFITS
#prA, prB, prC, prD = mcFit4(cosfit, xdat1, ydat1, xerr1, yerr1)
#print("COSFIT: ",prA, prB, prC, prD)

##LOGFIT
#log_prA, log_prB, log_prC, log_prD = mcFit4(logfit, xdat1, ydat1, xerr1, yerr1)
#print("LOGFIT: ",log_prA, log_prB, log_prC, log_prD)

##GAUSIAN FIT
#gausian_prA, gausian_prB = mcFit2(gausian, xdat1, ydat1, xerr1, yerr1)
#print("GAUSIAN: ", gausian_prA, gausian_prB)

##LORENTZIAN-CAUCHY FIT
#lc_prA, lc_prB, lc_prC = mcFit3(lorentzian_cauchy, xdat1, ydat1, xerr1, yerr1)#, p0=[0,633.7034807407312,3494.755148226808])
#print("LORENTZIAN-CAUCHY: ", lc_prA, lc_prB, lc_prC)

##SIGMOID FIT
#sig_prA, sig_prB, sig_prC, sig_prD = mcFit4(sigmoid, xdat1, ydat1, xerr1, yerr1)
#print("SIGMOID: ", sig_prA, sig_prB, sig_prC, sig_prD)

# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

# PLOT DATA AND ERROR BARS
ax1.scatter(xdat1,ydat1,s=1,c='b',alpha=0.1)
ax1.errorbar(xdat1, ydat1, yerr=yerr1, ecolor='grey',fmt='none', capsize=5, zorder=0)

x_plot = np.linspace(0,50,1000) # X-PLOTING FOR FITS

##linear fit
#ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')
ax1.plot(x_plot,linear(x_plot, (-35/50), 45), label='linear: FIT')

##poly2 fit
#ax1.plot(x_plot,poly2(x_plot, poly2_prA, poly2_prB, poly2_prC), label='poly2: FIT')

##poly3 fit
#ax1.plot(x_plot,poly3(x_plot, poly3_prA, poly3_prB, poly3_prC, poly3_prD), label='poly3: FIT')

##exponential fit
#ax1.plot(x_plot,exponential(x_plot, exp_prA, exp_prB, exp_prC, exp_prD), label='exponential: FIT')

##cosfit
#ax1.plot(x_plot,cosfit(x_plot, prA, prB, prC, prD), label='cosfit: FIT')

##logfit 
#ax1.plot(x_plot,logfit(x_plot, log_prA, log_prB, log_prC, log_prD), label='logfit: FIT')

##guasian fit
#ax1.plot(x_plot,gausian(x_plot, gausian_prA, gausian_prB), label='gausian: FIT')

##lorentzian-cauchy fit
#ax1.plot(x_plot,lorentzian_cauchy(x_plot, lc_prA, lc_prB, lc_prC), label='lorentzian-cauchy: FIT')

##sigmoid fit
#ax1.plot(x_plot,sigmoid(x_plot, sig_prA, sig_prB, sig_prC, sig_prD), label='sigmoid: FIT')



ax1.set_xlabel('$NVISITS$', fontsize=18)
ax1.set_ylabel('$VSCATTER$', fontsize=18)
plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
##PART 7
#Fit {\bf Fe_H, (Fe_H_ERR) vs. NI_FE (NI_FE_ERR)

temp_mask1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['Fe_H'] > -9000) & (star['NI_FE'] > -9000)
mask1 = np.where(temp_mask1)[0]

xdat1 = star['Fe_H'][mask1]
ydat1 = star['NI_FE'][mask1]
xerr1 = star['Fe_H_ERR'][mask1]
yerr1 = star['NI_FE_ERR'][mask1]

## LINEAR FIT
slope, intercept = mcFit2(linear, xdat1, ydat1, xerr1, yerr1)
print("LINEAR: ", slope, intercept)

## POLY2
#poly2_prA, poly2_prB, poly2_prC = mcFit3(poly2, xdat1, ydat1, xerr1, yerr1)
#print("POLY2: ", poly2_prA, poly2_prB, poly2_prC)

## POLY3
#poly3_prA, poly3_prB, poly3_prC, poly3_prD = mcFit4(poly3, xdat1, ydat1, xerr1, yerr1)
#print("POLY3: ", poly3_prA, poly3_prB, poly3_prC, poly3_prD)

## EXPONENTIAL
#exp_prA, exp_prB, exp_prC, exp_prD = mcFit4(exponential, xdat1, ydat1, xerr1, yerr1)
#print("EXPONENTIAL: ", exp_prA, exp_prB, exp_prC, exp_prD)

## COSFITS
#prA, prB, prC, prD = mcFit4(cosfit, xdat1, ydat1, xerr1, yerr1)
#print("COSFIT: ",prA, prB, prC, prD)

##LOGFIT
#log_prA, log_prB, log_prC, log_prD = mcFit4(logfit, xdat1, ydat1, xerr1, yerr1)
#print("LOGFIT: ",log_prA, log_prB, log_prC, log_prD)

##GAUSIAN FIT
#gausian_prA, gausian_prB = mcFit2(gausian, xdat1, ydat1, xerr1, yerr1)
#print("GAUSIAN: ", gausian_prA, gausian_prB)

##LORENTZIAN-CAUCHY FIT
#lc_prA, lc_prB, lc_prC = mcFit3(lorentzian_cauchy, xdat1, ydat1, xerr1, yerr1)#, p0=[0,633.7034807407312,3494.755148226808])
#print("LORENTZIAN-CAUCHY: ", lc_prA, lc_prB, lc_prC)

##SIGMOID FIT
#sig_prA, sig_prB, sig_prC, sig_prD = mcFit4(sigmoid, xdat1, ydat1, xerr1, yerr1)
#print("SIGMOID: ", sig_prA, sig_prB, sig_prC, sig_prD)

# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

# PLOT DATA AND ERROR BARS
ax1.scatter(xdat1,ydat1,s=1,c='b',alpha=0.1)
ax1.errorbar(xdat1, ydat1, yerr=yerr1, ecolor='grey',fmt='none', capsize=5, zorder=0)

x_plot = np.linspace(-2.0,0.75,1000) # X-PLOTING FOR FITS

##linear fit
ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')

##poly2 fit
#ax1.plot(x_plot,poly2(x_plot, poly2_prA, poly2_prB, poly2_prC), label='poly2: FIT')

##poly3 fit
#ax1.plot(x_plot,poly3(x_plot, poly3_prA, poly3_prB, poly3_prC, poly3_prD), label='poly3: FIT')

##exponential fit
#ax1.plot(x_plot,exponential(x_plot, exp_prA, exp_prB, exp_prC, exp_prD), label='exponential: FIT')

##cosfit
#ax1.plot(x_plot,cosfit(x_plot, prA, prB, prC, prD), label='cosfit: FIT')

##logfit 
#ax1.plot(x_plot,logfit(x_plot, log_prA, log_prB, log_prC, log_prD), label='logfit: FIT')

##guasian fit
#ax1.plot(x_plot,gausian(x_plot, gausian_prA, gausian_prB), label='gausian: FIT')

##lorentzian-cauchy fit
#ax1.plot(x_plot,lorentzian_cauchy(x_plot, lc_prA, lc_prB, lc_prC), label='lorentzian-cauchy: FIT')

##sigmoid fit
#ax1.plot(x_plot,sigmoid(x_plot, sig_prA, sig_prB, sig_prC, sig_prD), label='sigmoid: FIT')



ax1.set_xlabel('$Fe_H$', fontsize=18)
ax1.set_ylabel('$NI_{FE}$', fontsize=18)
plt.legend(loc='best', fontsize=18)
plt.show()